<a href="https://colab.research.google.com/github/Afaq0456/Fake-News/blob/main/Fakenews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
df = pd.read_csv('news_articles.csv')


In [ ]:
print(df.head())

                 author                      published  \
0     Barracuda Brigade  2016-10-26T21:41:00.000+03:00   
1  reasoning with facts  2016-10-29T08:47:11.259+03:00   
2     Barracuda Brigade  2016-10-31T01:41:49.479+02:00   
3                Fed Up  2016-11-01T05:22:00.000+02:00   
4                Fed Up  2016-11-01T21:56:00.000+02:00   

                                               title  \
0  muslims busted they stole millions in govt ben...   
1  re why did attorney general loretta lynch plea...   
2  breaking weiner cooperating with fbi on hillar...   
3  pin drop speech by father of daughter kidnappe...   
4  fantastic trumps  point plan to reform healthc...   

                                                text language  \
0  print they should pay all the back all the mon...  english   
1  why did attorney general loretta lynch plead t...  english   
2  red state  \nfox news sunday reported this mor...  english   
3  email kayla mueller was a prisoner and torture...  

In [ ]:

print(df.info())



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2096 entries, 0 to 2095
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   author                   2096 non-null   object 
 1   published                2096 non-null   object 
 2   title                    2096 non-null   object 
 3   text                     2050 non-null   object 
 4   language                 2095 non-null   object 
 5   site_url                 2095 non-null   object 
 6   main_img_url             2095 non-null   object 
 7   type                     2095 non-null   object 
 8   label                    2095 non-null   object 
 9   title_without_stopwords  2094 non-null   object 
 10  text_without_stopwords   2046 non-null   object 
 11  hasImage                 2095 non-null   float64
dtypes: float64(1), object(11)
memory usage: 196.6+ KB
None


In [ ]:


print(df.isnull().sum())



author                      0
published                   0
title                       0
text                       46
language                    1
site_url                    1
main_img_url                1
type                        1
label                       1
title_without_stopwords     2
text_without_stopwords     50
hasImage                    1
dtype: int64


In [ ]:

print(df.describe())


          hasImage
count  2095.000000
mean      0.777088
std       0.416299
min       0.000000
25%       1.000000
50%       1.000000
75%       1.000000
max       1.000000


In [ ]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

In [ ]:

# Drop rows with missing values in the 'text' column
df = df.dropna(subset=['text'])
df['title_without_stopwords'] = df['title_without_stopwords'].fillna('')
df['text_without_stopwords'] = df['text_without_stopwords'].fillna('')

# Confirm that there are no more missing values
print("\nAfter handling missing values:")
print(df.isnull().sum())


After handling missing values:
author                     0
published                  0
title                      0
text                       0
language                   0
site_url                   0
main_img_url               0
type                       0
label                      0
title_without_stopwords    0
text_without_stopwords     0
hasImage                   0
cleaned_text               0
dtype: int64


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')


# Function to perform text preprocessing
def preprocess_text(text):
    # Text cleaning
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub('[^a-zA-Z]', ' ', text)  # Remove special characters and numbers
    text = re.sub(r'\s+', ' ', text)  # Remove extra whitespaces

    # Tokenization
    words = word_tokenize(text)

    # Stopword removal
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word.lower() not in stop_words]

    # Normalization and stemming
    ps = PorterStemmer()
    words = [ps.stem(word.lower()) for word in words]

    # Removal of non-textual elements
    words = [word for word in words if word.isalpha()]

    # Join the processed words back into a cleaned text
    cleaned_text = ' '.join(words)

    return cleaned_text

# Apply preprocessing to the 'text' column
df['cleaned_text'] = df['text'].apply(preprocess_text)

# Save the cleaned dataset to a new CSV file
df.to_csv('cleaned_news_articles.csv', index=False)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Split the data into features (X) and labels (y)
X = df['cleaned_text']
y = df['label'].apply(lambda x: 0 if x == 'Real' else 1)

# Split the data into training (80%) and testing (20%) sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# TF-IDF Vectorization
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Model 1: k-Nearest Neighbors (kNN)
knn_model = KNeighborsClassifier()

# Hyperparameter Optimization for kNN
knn_param_grid = {'n_neighbors': [3, 5, 7],
                  'weights': ['uniform', 'distance']}
knn_grid_search = GridSearchCV(knn_model, knn_param_grid, cv=3)
knn_grid_search.fit(X_train_tfidf, y_train)
best_knn_model = knn_grid_search.best_estimator_

# Retrain kNN model with optimized hyperparameters
best_knn_model.fit(X_train_tfidf, y_train)

# Predictions
knn_train_preds = best_knn_model.predict(X_train_tfidf)
knn_test_preds = best_knn_model.predict(X_test_tfidf)

# Evaluate kNN model
print("\nOptimized kNN Accuracy on Training Set:", accuracy_score(y_train, knn_train_preds))
print("Optimized kNN Accuracy on Test Set:", accuracy_score(y_test, knn_test_preds))
print("\nOptimized kNN Classification Report on Test Set:")
print(classification_report(y_test, knn_test_preds))

# Model 2: Support Vector Machine (SVM)
svm_model = SVC()

# Hyperparameter Optimization for SVM
svm_param_grid = {'C': [0.1, 1, 10],
                  'kernel': ['linear', 'rbf']}
svm_grid_search = GridSearchCV(svm_model, svm_param_grid, cv=3)
svm_grid_search.fit(X_train_tfidf, y_train)
best_svm_model = svm_grid_search.best_estimator_

# Retrain SVM model with optimized hyperparameters
best_svm_model.fit(X_train_tfidf, y_train)

# Predictions
svm_train_preds = best_svm_model.predict(X_train_tfidf)
svm_test_preds = best_svm_model.predict(X_test_tfidf)

# Evaluate SVM model
print("\nOptimized SVM Accuracy on Training Set:", accuracy_score(y_train, svm_train_preds))
print("Optimized SVM Accuracy on Test Set:", accuracy_score(y_test, svm_test_preds))
print("\nOptimized SVM Classification Report on Test Set:")
print(classification_report(y_test, svm_test_preds))

# Model 3: Naive Bayes (Multinomial)
nb_model = MultinomialNB()

# Hyperparameter Optimization for Naive Bayes
nb_param_grid = {'alpha': [0.1, 0.5, 1.0, 1.5, 2.0]}
nb_grid_search = GridSearchCV(nb_model, nb_param_grid, cv=3)
nb_grid_search.fit(X_train_tfidf, y_train)
best_nb_model = nb_grid_search.best_estimator_

# Retrain Naive Bayes model with optimized hyperparameters
best_nb_model.fit(X_train_tfidf, y_train)

# Predictions
nb_train_preds = best_nb_model.predict(X_train_tfidf)
nb_test_preds = best_nb_model.predict(X_test_tfidf)

# Evaluate Naive Bayes model
print("\nOptimized Naive Bayes Accuracy on Training Set:", accuracy_score(y_train, nb_train_preds))
print("Optimized Naive Bayes Accuracy on Test Set:", accuracy_score(y_test, nb_test_preds))
print("\nOptimized Naive Bayes Classification Report on Test Set:")
print(classification_report(y_test, nb_test_preds))



Optimized kNN Accuracy on Training Set: 0.9975609756097561
Optimized kNN Accuracy on Test Set: 0.6829268292682927

Optimized kNN Classification Report on Test Set:
              precision    recall  f1-score   support

           0       0.61      0.54      0.57       161
           1       0.72      0.78      0.75       249

    accuracy                           0.68       410
   macro avg       0.67      0.66      0.66       410
weighted avg       0.68      0.68      0.68       410


Optimized SVM Accuracy on Training Set: 0.9353658536585366
Optimized SVM Accuracy on Test Set: 0.7414634146341463

Optimized SVM Classification Report on Test Set:
              precision    recall  f1-score   support

           0       0.74      0.53      0.62       161
           1       0.74      0.88      0.81       249

    accuracy                           0.74       410
   macro avg       0.74      0.70      0.71       410
weighted avg       0.74      0.74      0.73       410


Optimized Naive